# Prediction de l'energie des molécules

#### imporation des librairies

In [1]:
import pandas as pd
import numpy as np
import scipy, matplotlib
from scipy.fft import fft
import plotly.express as px
# ase est une librairie pour lire les fichiers .xyz (souvent utilisé pour representer des atomes.) site : https://wiki.fysik.dtu.dk/
from ase.io.xyz import read_xyz
from ase.io import read
from ase.visualize import view
import os 
from dscribe.descriptors.coulombmatrix import CoulombMatrix
from dscribe.descriptors.sinematrix import SineMatrix
from dscribe.descriptors.ewaldsummatrix import EwaldSumMatrix
from dscribe.descriptors.mbtr import MBTR
# librairies de data science.
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [3]:
import pywt

In [4]:
from scipy.signal import cwt

In [5]:
cA, cD = pywt.dwt([1, 2, 3, 4], 'db1')

In [6]:
StandardScaler().fit_transform(np.array([1,2,3,4]).reshape(-1, 1)).flatten()

array([-1.34164079, -0.4472136 ,  0.4472136 ,  1.34164079])

In [7]:
l_atoms = [ 'H','C','O','N','S','Cl']

### Prise en main de la librairie ase

Dans cette partie, nous prenons en main la librairie ASE et regardons les différentes fonctions qui pourrait nous être utile pour la suite.

In [ ]:
test = read("./train/atoms/train/id_1.xyz")

In [ ]:
view(test, viewer='x3d')

In [ ]:
from ase.calculators.abinit import Abinit

In [ ]:
calc = Abinit()

In [ ]:
test.calc = calc

In [ ]:
test.get_initial_charges()

In [16]:
import numpy as np
from dscribe.descriptors import MBTR

# Setup : best = 5
def getMBTR(N=5) :
    mbtr = MBTR(
        species=l_atoms,
        k1={
            "geometry": {"function": "atomic_number"},
            "grid": {"min": 0, "max": 8, "n": N, "sigma": 0.1},
        },
        k2={
            "geometry": {"function": "inverse_distance"},
            "grid": {"min": 0, "max": 1, "n": N, "sigma": 0.1},
            "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
        },
        k3={
            "geometry": {"function": "cosine"},
            "grid": {"min": -1, "max": 1, "n": N, "sigma": 0.1},
            "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
        },
        periodic=False,
        normalization="l2_each",
    )
    return mbtr

In [ ]:
mbtr = getMBTR(N=5)
mbtr_test = mbtr.create(test)

#print(mbtr_test)
print(mbtr_test.shape)


In [ ]:
from ase.build import molecule

water = molecule("H2O")

# Create MBTR output for the system
mbtr_water = mbtr.create(water)

print(mbtr_water)
print(mbtr_water.shape)

In [ ]:
px.line(mbtr_water)

In [ ]:
px.line(mbtr_test)

#### Listing des infos (fonctions) complémentaires potentiellement interessantes pour une molécule

In [ ]:
d_tab1 = test.get_all_distances()

In [ ]:
test.euler_rotate(phi=2.0, theta=8.0, psi=5.0, center=(0, 0, 0))

In [ ]:
d_tab2 = test.get_all_distances()

In [ ]:
d_tab1

In [ ]:
d_tab2

In [ ]:
d_tab1[0,2]

In [ ]:
d_tab2[0,2]

In [ ]:
np.equal(d_tab1,d_tab2)

In [ ]:
test.get_masses()#.sum()

In [ ]:
test.get_chemical_formula(mode='metal', empirical=False)

In [ ]:
test.get_atomic_numbers()

In [ ]:
pd.Series(test.get_chemical_symbols()).value_counts()

In [ ]:
test.get_global_number_of_atoms()

In [ ]:
np.linalg.norm(test.get_positions())

In [ ]:
np.linalg.norm(test.get_positions())

In [ ]:
#test.get_total_energy()

In [ ]:
test.get_kinetic_energy()

In [ ]:
test.set_scaled_positions(test.get_scaled_positions())

In [ ]:
test.get_scaled_positions()

In [ ]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_charges()
except Exception as e: 
    print(e)
    print("Erreur : Necessite de donner un calculateur à la molécule")

In [ ]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_dipole_moment()
except : 
    print("Erreur : Necessite de donner un calculateur à la molécule")

In [ ]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_forces(apply_constraint=True, md=False)
except : 
    print("Erreur : Necessite de donner un calculateur à la molécule")

In [ ]:
nb_atoms = test.get_global_number_of_atoms()

In [ ]:
test.get_moments_of_inertia(vectors=False)

In [ ]:
test.get_center_of_mass()

In [ ]:
CM = CoulombMatrix(test.get_global_number_of_atoms(), permutation='none', sigma=None, seed=None, flatten=True, sparse=False)
cm = CM.create(test, n_jobs=1, verbose=False)

In [ ]:
np.array_equal(cm,cm_2)

In [ ]:
cm_2 = CM.create(test, n_jobs=1, verbose=False)

In [ ]:
# checking if both the arrays are of equal size
a =cm
b = a.transpose()
if a.shape == b.shape:
    # comparing the arrays using == and all() method
    if (a == b).all():
        print("The Array or Matrix is Symmetric")
    else:
        print("The Array / Matrix is Not Symmetric")
else:
    print("The Array / Matrix is Not Symmetric")

In [ ]:
pd.DataFrame(a)

In [ ]:
from ase.build import molecule

# Molecule created as an ASE.Atoms
methanol = molecule("CH3OH")

In [ ]:
cm = CoulombMatrix(
    n_atoms_max=6,
    flatten=False,
    permutation="sorted_l2"
)

# Translation
methanol.translate((5, 7, 9))
cm_methanol = cm.create(methanol)
print(cm_methanol)

# Rotation
methanol.rotate(90, 'z', center=(0, 0, 0))
cm_methanol = cm.create(methanol)
print(cm_methanol)

# Permutation
upside_down_methanol = methanol[::-1]
cm_methanol = cm.create(upside_down_methanol)
print(cm_methanol)

In [ ]:
L = []
nb_atoms_max = 23
nb_atoms = methanol.get_global_number_of_atoms()
for i in range(nb_atoms) :
    L+= list(cm_methanol[i,0:i+1])
    
taille = 0 
for i in range(nb_atoms_max):
    taille+= nb_atoms_max - i
    
L+= [0]*(taille-len(L))

### Chargement des données et mise en forme

In [8]:
df_train_energies = pd.read_csv("./train/energies/train.csv",sep=",",dtype = {"id":"str"})

In [9]:
# Dictionnaire regroupant les différents atomes.
def get_dict_atoms(path) :
    """
    input : path of the file
    return : dictionnary of atoms
    """
    dict_atoms_train = {}
    l = os.scandir(path)
    l = [e.name for e in l]
    for e in l : 
        dict_atoms_train[e.split(".")[0].split("_")[1]] = read(path+e)
    return dict_atoms_train

In [10]:
dict_atoms_train = get_dict_atoms("./train/atoms/train/")
dict_atoms_test = get_dict_atoms("./train/atoms/test/")

In [11]:
pct_val = 0.33

In [12]:
import random

In [13]:
### Fonctions utiles :
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [17]:
def getMBTR(N=5) :
    mbtr = MBTR(
        species=l_atoms,
        k1={
            "geometry": {"function": "atomic_number"},
            "grid": {"min": 0, "max": 8, "n": N, "sigma": 0.1},
        },
        k2={
            "geometry": {"function": "inverse_distance"},
            "grid": {"min": 0, "max": 1, "n": N, "sigma": 0.1},
            "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
        },
        k3={
            "geometry": {"function": "cosine"},
            "grid": {"min": -1, "max": 1, "n": N, "sigma": 0.1},
            "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
        },
        periodic=False,
        normalization="l2_each",
    )
    return mbtr

In [18]:
# require getMBTR
def getDf(dict_atoms_train, permutation="sorted_l2", sigma=None,type_d = "Train",transform =("autre",None),representation="mbtr") : #,"sorted_l2","random","eigenspectrum"
    df_train = pd.DataFrame({"id":dict_atoms_train.keys(),"Molecule":dict_atoms_train.values()})
    def getVectorisedMolecule(x,version = "mbtr") :
        if version == "CM" : # matrice de coulomb
            CM = CoulombMatrix(nb_max_atoms, permutation=permutation, sigma=sigma, seed=None, flatten=True, sparse=False)
            # vectorised coulomb matrix.
            vcm = np.array(CM.create(x["Molecule"],verbose=False))
            return vcm
        else : # mbtr 
            mbtr = getMBTR(N=5)
            # vectorised many body tensor representation.
            vmbtr = mbtr.create(x["Molecule"]).flatten()
            return vmbtr #StandardScaler().fit_transform(cm.reshape(-1, 1)).flatten()
    def getParams(x) :
        return x.get_chemical_formula(mode='hill', empirical=False),x.get_global_number_of_atoms(),x.get_masses().sum()
    def getDwt(x,representation) :
        cA,cB = pywt.dwt(getVectorisedMolecule(x,representation),transform[1])
        to_return = list(cA)+list(cB)
        
        assert len(cA)+len(cB)==len(to_return)
        return list(cA)
    df_train[["Molecule_formula","Number_of_atoms","Masse"]] = None
    df_train[["Molecule_formula","Number_of_atoms","Masse"]] = df_train.apply(lambda x : getParams(x["Molecule"]), axis=1,result_type='expand')
    sub = df_train["Molecule"].apply(lambda x : pd.Series(x.get_chemical_symbols()).value_counts())
    sub.fillna(0,inplace = True)
    col_to_add = sub.columns
    df_train[col_to_add]= sub
    nb_max_atoms = df_train["Number_of_atoms"].max()
    if transform[0] == "fft":
        sub = df_train.apply(lambda x : np.abs(fft(getVectorisedMolecule(x,representation))), axis=1,result_type='expand').copy()
    elif transform[0] =="cwt" :
        sub = df_train.apply(lambda x : getDwt(x,representation) , axis=1,result_type='expand').copy()
    else :
        sub = df_train.apply(lambda x : getVectorisedMolecule(x,representation), axis=1,result_type='expand').copy()

    df_train[sub.columns] = sub
    if type_d == "Train" :
        df_train = df_train.merge(df_train_energies,on="id",how="left")
        df_train.sort_values("id",inplace = True)
        df_train.dropna(inplace=True)
    return df_train,sub

In [19]:
# nombre de molécule uniques 
transform=(None,None)
representation = "mbtr"
df_train,_ =getDf(dict_atoms_train,permutation=None,sigma=None,transform=transform,representation=representation)
df_test,_  =getDf(dict_atoms_test,permutation=None,sigma=None,transform=transform,type_d="Test",representation=representation)

/home/n7student/miniconda3/envs/forDataScience/lib/python3.9/site-packages/pandas/core/frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


In [ ]:
df_test.to_csv("process_data_"+transform[1]+"_test.csv",sep=";",header=True,encoding="utf-8-sig",index=False)
df_train.to_csv("process_data_"+transform[1]+".csv",sep=";",header=True,encoding="utf-8-sig",index=False)

### for Data Augmentation 

In [ ]:
df_train_all = df_train.copy()
df_train = df_train_all.sample(frac = 0.6)
df_val = df_train_all.drop(df_train.index)
l_id_train = list(df_train["id"].unique())
dict_atoms_sub_train = {}
for elt in l_id_train :
    dict_atoms_sub_train[elt] = dict_atoms_train[elt]
## Data augmenation
Nb_aug = 5
for i in range(Nb_aug) :
    df,_ = getDf(dict_atoms_sub_train,permutation="random",sigma=0.5,transform=transform)
    df_train = pd.concat([df_train,df],ignore_index=True)

In [ ]:
C_M = df_train[[e for e in range(763)]]#df_train[(df_train["Molecule_formula"]=="C6H9N")]

In [ ]:
C_M = C_M.transpose()
# X = C_M.values
# X= StandardScaler().fit_transform(X)
# nC_M = pd.DataFrame(data=X,columns=C_M.columns)

In [ ]:
px.line(C_M)

In [ ]:
df_train_energies.describe()

In [ ]:
px.histogram(df_train_energies["energy"].values)

In [ ]:
df_train.info()

In [ ]:
df_train.shape

In [ ]:
print("Nombre de molécules différentes : "+str(len(df_train["Molecule_formula"].unique())))

In [ ]:
print("Nombre de molécules différentes : "+str(len(df_test["Molecule_formula"].unique())))

In [ ]:
if set(df_test["Molecule_formula"].unique())==set(df_train["Molecule_formula"].unique()) :
    print("Les molécules présentes dans l'ensemble test sont les même que ceux présents dans l'ensemble d'entrainement")
else :
    print("Attention l'ensemble des molécules présente dans le jeu d'entrainement et de test est différent")
    

In [ ]:
px.scatter(df_train,x="Number_of_atoms",y="energy",hover_data = ["energy","Molecule_formula"])

In [ ]:
px.scatter(df_train,x="Masse",y="energy",hover_data = ["energy","Molecule_formula"])

In [ ]:
list(df_train.columns)

### Represention de la matrice de coulomb tel qu'il soit invariant par translation ,rotation et permutations

In [ ]:
Y_cm = df_train["Molecule_formula"].values

In [ ]:
X_cm = sub.values

In [ ]:
X_cm = StandardScaler().fit_transform(X_cm)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cm, Y_cm, test_size=0.2, random_state=42)

#### Reduction de la dimension par lda

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
clf = LinearDiscriminantAnalysis(n_components=7)
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_train,y_train)

In [ ]:

X_new = clf.fit_transform(X_train,y_train)

In [ ]:
X_new[:,0]

In [ ]:
px.scatter(x=X_new[:,0],y=X_new[:,1],color=y_train)

### Entrainement du modèle.

#### Définition de la métrique d'évaluation.

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

#### Création d'un jeu de données d'entrainement et de test  / Scaling.

In [ ]:
Y = df_train["energy"].values

In [ ]:
to_drop = ["id","Molecule","Molecule_formula","energy"]

In [ ]:
X = df_train.drop(to_drop,axis=1)

In [ ]:
scaler = StandardScaler()
scaler.fit(X.values)
X_ = scaler.transform(X.values)

In [22]:
if False : 
    X_val = df_val.drop(to_drop,axis=1)
    X_val_ = scaler.transform(X_val.values)
    Y_val= df_val["energy"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, Y, test_size=0.1, random_state=42)

In [ ]:
X_train.shape

### XGboost

In [ ]:
import xgboost as xg

In [ ]:
xgb_r = xg.XGBRegressor(eval_metric="rmse",max_depth=8,n_estimators=1000,reg_alpha=0.2)#reg_alpha = 2.5,reg_lambda=2)
xgb_r.fit(X_train, np.array(y_train))

In [ ]:
xgb_r.score(X_train,y_train)

In [ ]:
xgb_r.score(X_test,y_test)

In [ ]:
y_pred = xgb_r.predict(X_test)

In [ ]:
rmse(y_test, y_pred)

In [ ]:
rmse(y_train, xgb_r.predict(X_train))

In [ ]:
rmse(xgb_r.predict(X_val_),Y_val)

In [ ]:
X_train.shape

### Random Forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_train,y_train)

In [ ]:
y_predicted_rf_train = regr.predict(X_train)

In [ ]:
y_predicted_rf_test = regr.predict(X_test)

In [ ]:
rmse(y_predicted_rf_test ,y_test)

In [ ]:
rmse(y_predicted_rf_train ,y_train)

In [ ]:
rmse(regr.predict(X_val_),Y_val)

### SVM

In [ ]:
from sklearn import svm

In [ ]:
regr_svm = svm.SVR()
regr_svm.fit(X_train,y_train)

In [ ]:
regr_svm.score(X_train,y_train)

In [ ]:
y_predicted_svm_train = regr_svm.predict(X_train)

In [ ]:
y_predicted_svm_test= regr_svm.predict(X_test)

In [ ]:
rmse(y_predicted_svm_test ,y_test)

### SGD_REGRESSOR

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
regr_sgdr = SGDRegressor()
regr_sgdr.fit(X_train,y_train)

In [ ]:
regr_sgdr.score(X_train,y_train)

In [ ]:
y_predicted_sgdr_train = regr_sgdr.predict(X_train)

In [ ]:
y_predicted_sgdr_test = regr_sgdr.predict(X_test)

In [ ]:
rmse(y_predicted_sgdr_test ,y_test)

### Linear Lasso

In [ ]:
from sklearn import linear_model

In [ ]:
reg_lasso = linear_model.Lasso(alpha=0.08)
reg_lasso.fit(X_train,y_train)

In [ ]:
reg_lasso.score(X_train,y_train)

In [ ]:
rmse(reg_lasso.predict(X_train),y_train)

In [ ]:
rmse(reg_lasso.predict(X_test),y_test)

### Ridge

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge= Ridge(alpha=0.8)
ridge.fit(X_train,y_train)

In [ ]:
ridge.score(X_train,y_train)

In [ ]:
rmse(ridge.predict(X_train),y_train)

In [ ]:
rmse(ridge.predict(X_test),y_test)

In [ ]:
#rmse(ridge.predict(X_val_),Y_val)

### CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Librairies et fonctions nécessaires au design des réseaux de neurones
# import keras
# from keras import layers
# from keras import models
from tensorflow.keras.layers import Dense,Conv1D,Conv2D, MaxPooling1D, Flatten, Activation,Dropout
from tensorflow.keras.models import Model, Sequential

In [ ]:
scaler_MinMax = preprocessing.MinMaxScaler()#QuantileTransformer(output_distribution="normal",random_state=0)
scaler_MinMax.fit(X.values)

In [ ]:
X_cnn = scaler.transform(X.values)#

In [ ]:
X_cnn = X_cnn.reshape(X_cnn.shape[0], X_cnn.shape[1], 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     X_cnn, Y, test_size=0.10, random_state=42)

In [ ]:
X_train.shape

In [ ]:
import tensorflow.keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
model = Sequential()
# model.add(Conv1D(32, 2, activation="relu", input_shape=(537,1)))
# model.add(Flatten())
# model.add(Dense(1000, activation="relu"))
# model.add(Dense(500, activation="relu"))
model.add(Dense(124, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(loss=root_mean_squared_error, optimizer="adam",metrics=["mse",root_mean_squared_error])
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss",patience=15)
history = model.fit(X_train, y_train,validation_split = 0.1,batch_size=7,epochs=7)#,callbacks=[early_stop])

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error ')
    plt.legend()
    plt.grid(True)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plot_loss(history)

In [ ]:
y_pred_cnn = model.predict(X_test)

In [ ]:
rmse(y_pred_cnn,y_pred)

### Bayesian neural network

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow_datasets as tfds

In [ ]:
hidden_units = [18, 9, 3]
learning_rate = 0.001

In [ ]:
##### Méthode 1

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(pd.DataFrame(data=X_train,columns=X.columns.astype(str))), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(pd.DataFrame(data=X_test,columns=X.columns.astype(str))), y_test))

In [ ]:
train_dataset.element_spec

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [ ]:
def create_model_inputs(FEATURE_NAMES):
    inputs = {}
    for feature_name in FEATURE_NAMES:
#         if type(column[0]) == str:
#             dtype = tf.string
#         elif (name in categorical_feature_names or \
#             name in binary_feature_names):
#             dtype = tf.int64
#         else:
#             dtype = tf.float32
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [ ]:
def create_baseline_model(Feature):
    inputs = create_model_inputs(Feature)
    input_values = [value for _, value in inputs.items()]
    features = keras.layers.concatenate(input_values)
    #features = layers.BatchNormalization()(inputs)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
X_train.shape

In [ ]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model(X.columns.astype(str))
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

## Submission

In [ ]:
X_submit = df_test.drop(["id","Molecule","Molecule_formula"],axis=1)

In [ ]:
X_submit_ = scaler.transform(X_submit.values)

In [ ]:
X_submit_.shape

In [ ]:
predicted = ridge.predict(X_submit_)

In [ ]:
df_pred = pd.DataFrame({"id":df_test["id"].values,"predicted":predicted})

In [ ]:
# df_pred = pd.DataFrame({"id":df_test["id"].values,"energy":predicted})
# dq = pd.read_csv("test.csv",sep=",")
# l_id = dq["id"].astype(str).values

In [ ]:
#l_id = dq["id"].astype(str).values

In [ ]:
df_pred.to_csv("sub1.csv",header=True,encoding="utf-8-sig",sep=",",index=False)